# Datathon (11/20/20 - 11/21/20)

In [61]:
from urllib.request import Request, urlopen
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bruce\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bruce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bruce\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
def get_random_ua():
    random_ua = ''
    ua_file = 'user-agents.txt'
    with open(ua_file) as f:
        lines = f.readlines()
        prng = np.random.RandomState()
        index = prng.permutation(len(lines) - 1)
        idx = np.asarray(index, dtype=np.integer)[0]
        random_ua = lines[int(idx)]
    return random_ua.replace('\n','')

In [63]:
def get_data(url, package):
    res = Request(url, headers={'User-Agent': get_random_ua()})
    try:
        raw_data = urlopen(res).read()
    except:
        print("Buffering...")
    s = raw_data.decode('utf-8')
    if package == 'json':
        return json.loads(s)
    elif package == 'html':
        return BeautifulSoup(s, features='html.parser')

In [70]:
stop_words = nltk.corpus.stopwords.words("english")

def tokenize(s):
    sentences = nltk.sent_tokenize(str(s))
    words = set()
    for sentence in sentences:
        for w in nltk.word_tokenize(sentence):
            words.add(w)
    global stop_words
    words = set([w for w in words if w not in stop_words])
    lmt = nltk.stem.WordNetLemmatizer()
    final = set()
    for w in words:
        final.add(lmt.lemmatize(w).lower())
    return final
    
understood_mission = '''Understood is a non-profit dedicated to serving the millions of families of kids who learn and think differently.

Since 2014, Understood has served millions of families of kids who learn and think differently. In 2012, our Workplace program was created and has helped thousands of people with all types of disabilities find meaningful employment at inclusive companies.

Our programs for Families, Educators, and Young Adults focus on empowering people who learn and think differently and those who support them, offering customized, accessible resources and a compassionate community. Our Workplace program helps organizations across industries create inclusive workplaces, by developing and implementing best-in-class disability inclusion programs so they can hire, advance, and retain people with disabilities.

We work with educators, health care professionals, researchers, and human resource professionals to provide proven, vetted information. By providing resources, support, and community, we can prevent people from being left behind and start to address systemic issues like high school dropout rates, underemployment, and the stigma surrounding disabilities.'''

understood_mission_words = tokenize(understood_mission)

In [71]:
def like_understood(mission):
    mission_words = tokenize(mission)
    score = 0
    global understood_mission_words
    for word in mission_words:
        if word in understood_mission_words:
            score += 1
    return score

In [66]:
# Propublica subsection codes
# (don't worry I automated this)
pro_subsection_codes = {'2': '501(c)(2)',
                        '3': '501(c)(3)',
                        '4': '501(c)(4)',
                        '5': '501(c)(5)',
                        '6': '501(c)(6)',
                        '7': '501(c)(7)',
                        '8': '501(c)(8)',
                        '9': '501(c)(9)',
                        '10': '501(c)(10)',
                        '11': '501(c)(11)',
                        '12': '501(c)(12)',
                        '13': '501(c)(13)',
                        '14': '501(c)(14)',
                        '15': '501(c)(15)',
                        '16': '501(c)(16)',
                        '17': '501(c)(17)',
                        '18': '501(c)(18)',
                        '19': '501(c)(19)',
                        '21': '501(c)(21)',
                        '22': '501(c)(22)',
                        '23': '501(c)(23)',
                        '25': '501(c)(25)',
                        '26': '501(c)(26)',
                        '27': '501(c)(27)',
                        '28': '501(c)(28)',
                        '92': '4947(a)(1)'}

In [67]:
# Construct the dataframe

df = pd.DataFrame(columns=['ein',
                           'name',
                           'IRS_sub_name',
                           'address',
                           'city',
                           'state',
                           'zipcode',
                           'subsection_code',
                           'ntee_code',
                           'site_url',
                           'mission',
                           'linkedin_link'])

In [68]:
i = 0

while i <= 3857:
    try:
        url = f'https://projects.propublica.org/nonprofits/api/v2/search.json?page={i}'
        data = get_data(url, 'json')
        orgs = data['organizations']
        for org in orgs:
            org_url = f'https://projects.propublica.org/nonprofits/api/v2/organizations/{org["ein"]}.json'
            org_data = get_data(org_url, 'json')['organization']
            org_df = {k: [org_data[k]] for k in org_data}
            org_df['subsection_code'] = [pro_subsection_codes[str(org_df['subsection_code'][0])]]
            org_df.update({'IRS_sub_name':[org['sub_name']]})

            dom = get_data(f'https://www.guidestar.org/profile/{org["strein"]}', 'html')

            mission = dom.select('p#mission-statement')
            if mission != []:
                s = mission[0].string
                if s == 'This organization has not provided GuideStar with a mission statement.':
                    mission = ''
                else:
                    mission = s
            else:
                mission = ''

            link = dom.select('div.description.pt-3>span.website>a')
            if link != []:
                site_url = link[0]['href']
            else:
                site_url = ''
            org_df.update({'site_url':[site_url], 'mission':[mission]})


            df = pd.concat([df, pd.DataFrame(org_df)], join = 'inner')
        
    except:
        time.sleep(60)
                       
    i += 1
    # interupt cell at anytime to get a partial dataframe

Buffering...
Buffering...
Buffering...
Buffering...
Buffering...
Buffering...


KeyboardInterrupt: 

In [72]:
df['like_understood'] = df['mission'].apply(like_understood)

In [73]:
display(df)

,ein,name,IRS_sub_name,address,city,state,zipcode,subsection_code,ntee_code,site_url,mission,like_understood
0,464744976,0 DEBT EDUCATION INC,0 DEBT EDUCATION INC,PO BOX 3777,SANTA ROSA,CA,95402-3777,501(c)(3),P51,,,0
0,272620044,0 TOLERANCE INC,0 TOLERANCE INC,2167 MERRYMOUNT DR,SUWANEE,GA,30024-3208,501(c)(3),B99,http://www.0ti.org/,The 0 Tolerance mission is to promote TECHNOCA...,3
0,814045228,0 U R PASSION,0 U R PASSION,6816 W 4TH AVE,KENNEWICK,WA,99336-9506,501(c)(3),L40,,PROVIDING HOUSING SUPPORT RECOV,2
0,824704419,00 MOVEMENT INC,00 MOVEMENT INC,713 CROOKED OAK DR,PENSACOLA,FL,32514-9541,501(c)(3),X20,http://www.00movement.org/,Contending for the ending of human trafficking...,2
0,226062777,00006 LOCAL,00006 LOCAL,608 E BALTIMORE PIKE,MEDIA,PA,19063-1735,501(c)(5),None,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,454145760,100 WHEELS INC,100 WHEELS INC,PO BOX 13686,ATLANTA,GA,30324-0686,501(c)(3),P82,,,0
0,134068307,100 WHO CARE INC,100 WHO CARE INC,27-08 40TH AVE 2ND FL,LONG ISLAND CITY,NY,11101-3725,501(c)(3),T30,,THE ORGANIZATIONS MISSION IS TO PROMOTE FURTHE...,1
0,851536666,1000 IN ACTION,1000 IN ACTION,PO BOX 24,FAIRFIELD,MT,59436-0024,501(c)(3),P12,,,0
0,820648835,1000 ISLANDS CHARITY POKER RUN CORP,1000 ISLANDS CHARITY POKER RUN CORP,PO BOX 56,CLAYTON,NY,13624-0056,501(c)(3),N12,http://www.1000islandsrun.com/,The 1000 Islands Charity Poker Run is a grassr...,9


In [74]:
df.to_csv(path_or_buf = 'datathon.csv', index = False)

In [ ]:
# not used

# def append_wiki(url):
#     return 'https://en.wikipedia.org/' + url
# def get_attr(attr, heads, dom, q):
#     if attr in t_heads:
#         site = dom.select(q)
#         if site != []:
#             return site[0]
#         else:
#             return ''
#     else:
#         return ''
# us_links = []
# while us_links == []:
#     dom = get_data('https://en.wikipedia.org/wiki/List_of_companies_of_the_United_States_by_state', 'html')
#     us_links = [append_wiki(a['href']) for a in dom.select('div[aria-labelledby="Lists_of_companies_by_U.S._state"] td.navbox-list li>a')]
#     for ul in us_links:
#         dom = get_data(ul, 'html')
#         st_links = [append_wiki(a['href']) for a in dom.select('h3 + ul li a')]
#         for sl in st_links:
#             dom = get_data(sl, 'html')
#             t_heads = [t.string for t in dom.select('table.infobox tbody th')]
#             if not('Defunct' in t_heads) and not(t_heads == []):
#                 name = dom.select('h1')[0].string.replace(' (Company)','').replace(' (company)','')
#                 for row in dom.select('table.infobox tr'):
#                     if 'Type' in str(row):
#                         print(row.select('a')[0].string)
                
                
#                 print(name)
#                 try:
#                     website = get_attr('Website', t_heads, dom, 'table.infobox tr>td a.external.text')['href']
#                 except:
#                     website = ''
#                 print(website)
#                 c_type = get_attr('Type', t_heads, dom, 'table.infobox ')
#                 # do some stuff
#                 # type, industry, headquarters, founded, key people, products, website